In [1]:
import boto3
from dotenv import load_dotenv
import os
load_dotenv()

USER_POOL_ID = os.getenv('USER_POOL_ID') 

# Reemplaza con tu User Pool ID

# Crear cliente de Cognito
client = boto3.client('cognito-idp')

# Obtener lista de usuarios
response = client.list_users(
    UserPoolId=USER_POOL_ID,
    Limit=60  # puedes ajustar este límite (máx 60 por llamada)
)

# Imprimir información de los usuarios
for user in response['Users']:
    print(f"Username: {user['Username']}")
    for attr in user['Attributes']:
        print(f"  {attr['Name']}: {attr['Value']}")
    print("-" * 20)


Username: 1205a484-30f1-70a2-4200-5120ef85a5f0
  email: alex.sopena.91@gmail.com
  email_verified: true
  name: Alex
  sub: 1205a484-30f1-70a2-4200-5120ef85a5f0
--------------------
Username: a2456424-e051-7005-ede4-48c7c1d51259
  email: fer@carazo.com
  email_verified: false
  name: Fer
  sub: a2456424-e051-7005-ede4-48c7c1d51259
--------------------
Username: b2a5f4c4-d021-7038-6dfd-661548fbb50d
  email: arrietaeguren@gmail.com
  email_verified: true
  name: Jarri
  sub: b2a5f4c4-d021-7038-6dfd-661548fbb50d
--------------------
Username: c2355484-2021-70d5-4010-174e31953a1a
  email: fernando.carazom@gmail.com
  email_verified: true
  name: Fernando
  sub: c2355484-2021-70d5-4010-174e31953a1a
--------------------
Username: c275d4d4-50c1-703b-8346-a62500e880e2
  email: mbrice@alumni.unav.es
  email_verified: true
  name: Maria 
  sub: c275d4d4-50c1-703b-8346-a62500e880e2
--------------------


In [2]:
from dotenv import load_dotenv
import os
# Load environment variables from a .env file
load_dotenv()
# Get the value of the environment variable
value = os.getenv("BREVO_TOKEN")

In [40]:
import brevo_python
from brevo_python.rest import ApiException
import pandas as pd

def brevo_get_all_emails_and_names_and_save_csv():
    """
    Fetches all emails from Brevo contacts, save them to a CSV file (nombre,email), and returns the list of emails
    Returns:
        list: A list of email addresses.
    """
    configuration = brevo_python.Configuration()
    configuration.api_key['api-key'] = os.getenv("BREVO_TOKEN")

    api_instance = brevo_python.ContactsApi(brevo_python.ApiClient(configuration))
    try:
        # Fetch all contacts
        api_response = api_instance.get_contacts(limit=1000)
        c = api_response.contacts
        # Prepare data for CSV
        data = pd.DataFrame({"emails": [x["email"] for x in c], "nombre": [x["attributes"]["NOMBRE"] for x in c]})
    except ApiException as e:
        print(f"Exception when calling ContactsApi->get_contacts: {e}")
        return []
    # Save to CSV
    data.to_csv("brevo_contacts.csv", index=False, encoding='utf-8')
    # Return the list of emails
    return data

In [41]:
c = brevo_get_all_emails_and_names_and_save_csv()

,emails,nombre
0,arrietaeguren@gmail.com,Jarri
1,fernando.carazo@veeva.com,Fer
2,fernando.carazom@gmail.com,fer


In [ ]:
from __future__ import print_function
import time
import brevo_python
from brevo_python.rest import ApiException
from pprint import pprint

# Configure API key authorization: api-key

# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
# configuration.api_key_prefix['api-key'] = 'Bearer'
# Configure API key authorization: partner-key
# configuration = brevo_python.Configuration()
# configuration.api_key['partner-key'] = 'YOUR_API_KEY'
# Uncomment below to setup prefix (e.g. Bearer) for API key, if needed
# configuration.api_key_prefix['partner-key'] = 'Bearer'

# create an instance of the API class
api_instance = brevo_python.AccountApi(brevo_python.ApiClient(configuration))

try:
    # Get your account information, plan and credits details
    api_response = api_instance.get_account()
    pprint(api_response)
except ApiException as e:
    print("Exception when calling AccountApi->get_account: %s\n" % e)


{'address': {'city': 'Pamplona',
             'country': 'Spain',
             'street': 'Casa',
             'zip_code': '31001'},
 'company_name': 'igles-ia',
 'email': 'fernando.carazom@gmail.com',
 'first_name': 'Fernando',
 'last_name': 'C.',
 'marketing_automation': None,
 'plan': [{'credits': 298.0,
           'credits_type': 'sendLimit',
           'end_date': None,
           'start_date': None,
           'type': 'free',
           'user_limit': None},
          {'credits': 0.0,
           'credits_type': 'sendLimit',
           'end_date': None,
           'start_date': None,
           'type': 'sms',
           'user_limit': None}],
 'relay': {'data': {'port': 587,
                    'relay': 'smtp-relay.brevo.com',
                    'user_name': '8d844b001@smtp-brevo.com'},
           'enabled': True}}


In [ ]:
api_instance = brevo_python.ContactsApi(brevo_python.ApiClient(configuration))

try:
    # Get the list of contacts
    api_response = api_instance.get_contacts()
except ApiException as e:
    print("Exception when calling ContactsApi->get_contacts: %s\n" % e)
    
api_response.contacts

contact_list = api_response.contacts
emails = [contact["email"] for contact in contact_list]
emails




['arrietaeguren@gmail.com',
 'fernando.carazo@veeva.com',
 'fernando.carazom@gmail.com']

In [4]:
from iglesia.utils import obtener_todos_los_textos
from crewai import LLM

LLM = LLM(
    model="gpt-4.1-nano",
    temperature=0.2,
    max_tokens=2000,
)


In [4]:
import re
def convertir_txt_a_html(path_txt):
    import re

    with open(path_txt, "r", encoding="utf-8") as f:
        raw_text = f.read()

    # Normalizar líneas: convertir todos los \r\n a \n
    raw_text = raw_text.replace("\r\n", "\n")

    # Dividir por bloques de texto separados por líneas vacías (aunque tengan espacios)
    bloques = re.split(r'\n\s*\n', raw_text.strip())

    html = ""
    for bloque in bloques:
        # Convertir URLs planas en enlaces HTML clicables
        bloque = re.sub(
            r'(https?://[^\s]+)',
            r'<a href="\1" style="color:#003366; text-decoration:none;" target="_blank">\1</a>',
            bloque.strip()
        )
        html += f"<p style='font-size:15px; color:#333;'>{bloque}</p>\n"

    return html


with open("../summaries-local/2025-05-19/resumen_semanal_igles-ia.txt", "r", encoding="utf-8") as f:
    resumen_txt = f.read()
    resumen_principal = convertir_txt_a_html(resumen_txt)

OSError: [Errno 63] File name too long: 'Resumen Semanal de Igles-IA:  \nEsta semana destacamos la homilía del Papa León XIV en la celebración del V Domingo de Pascua, donde inicia su pontificado enfatizando que el liderazgo en la Iglesia debe fundamentarse en el amor y la humildad, siguiendo el ejemplo de Pedro. El Papa subraya que la misión de la Iglesia es promover la unidad, la fraternidad y la reconciliación en un mundo marcado por la discordia y la violencia, siendo un fermento de paz, justicia social y amor de Dios. La caridad y el servicio son el corazón del Evangelio y la base para construir una sociedad más humana y solidaria. Este mensaje invita a todos los cristianos a vivir con amor y compromiso en la misión de la Iglesia, siendo testigos de la presencia de Dios en el mundo.  \n\nEnlaces a las fuentes originales:  \nhttps://www.vatican.va/content/leo-xiv/es/homilies/2025/documents/20250518-inizio-pontificato.html'

In [2]:
print(resumen_txt)

Resumen Semanal de Igles-IA:  
Esta semana destacamos la homilía del Papa León XIV en la celebración del V Domingo de Pascua, donde inicia su pontificado enfatizando que el liderazgo en la Iglesia debe fundamentarse en el amor y la humildad, siguiendo el ejemplo de Pedro. El Papa subraya que la misión de la Iglesia es promover la unidad, la fraternidad y la reconciliación en un mundo marcado por la discordia y la violencia, siendo un fermento de paz, justicia social y amor de Dios. La caridad y el servicio son el corazón del Evangelio y la base para construir una sociedad más humana y solidaria. Este mensaje invita a todos los cristianos a vivir con amor y compromiso en la misión de la Iglesia, siendo testigos de la presencia de Dios en el mundo.  

Enlaces a las fuentes originales:  
https://www.vatican.va/content/leo-xiv/es/homilies/2025/documents/20250518-inizio-pontificato.html


In [3]:
print(resumen_principal)

<p style='font-size:15px; color:#333;'>Resumen Semanal de Igles-IA:   Esta semana destacamos la homilía del Papa León XIV en la celebración del V Domingo de Pascua, donde inicia su pontificado enfatizando que el liderazgo en la Iglesia debe fundamentarse en el amor y la humildad, siguiendo el ejemplo de Pedro. El Papa subraya que la misión de la Iglesia es promover la unidad, la fraternidad y la reconciliación en un mundo marcado por la discordia y la violencia, siendo un fermento de paz, justicia social y amor de Dios. La caridad y el servicio son el corazón del Evangelio y la base para construir una sociedad más humana y solidaria. Este mensaje invita a todos los cristianos a vivir con amor y compromiso en la misión de la Iglesia, siendo testigos de la presencia de Dios en el mundo.</p>
<p style='font-size:15px; color:#333;'>Enlaces a las fuentes originales:   https://www.vatican.va/content/leo-xiv/es/homilies/2025/documents/20250518-inizio-pontificato.html</p>



In [ ]:
#  quiero que si es lunes 18, te quedes con las filas desde el lunes 11 a domingo 17

In [7]:
pd.Timestamp('today').normalize()

Timestamp('2025-05-18 00:00:00')

In [6]:
import pandas as pd

# Asumimos que 'df' es tu DataFrame cargado y tiene una columna 'fecha'
# Ejemplo de DataFrame:
data = {
    'fecha': ['2025-05-10', '2025-05-11', '2025-05-17', '2025-05-18', '2025-05-29'],
    'valor': [1, 2, 3, 4, 5]
}
df = pd.DataFrame(data)

# 1. Define el lunes actual (cuando se ejecuta el script)
# Si el script se ejecuta el lunes 19, run_date sería el lunes 19.
# Para simular que hoy es Lunes 19 de Mayo, 2025 (para este ejemplo):
# run_date = pd.Timestamp('2025-05-19').normalize()
# En tu script real, si se ejecuta un lunes, usarías:
run_date = pd.Timestamp('today').normalize()


# --- Lógica de filtrado en dos líneas ---
# Línea 1: Convierte tu columna 'fecha' original a formato datetime.
df['fecha_dt'] = pd.to_datetime(df['fecha'])

# Línea 2: Filtra el DataFrame para obtener la semana anterior completa (Lunes a Domingo).
# La semana anterior va desde (run_date - 7 días) hasta (run_date - 1 día), inclusive.
# Usamos '< run_date' para incluir todo el día anterior (domingo) hasta las 23:59:59.
df = df[(df['fecha_dt'] >= (run_date - pd.Timedelta(days=7))) & (df['fecha_dt'] < run_date)]
# --- Fin de la lógica ---

# Ahora 'df' contiene solo las filas desde el lunes anterior al domingo anterior.
print(f"Filtrando para la semana anterior al {run_date.strftime('%Y-%m-%d %A')}:")
print(df)

Filtrando para la semana anterior al 2025-05-18 Sunday:
        fecha  valor   fecha_dt
1  2025-05-11      2 2025-05-11
2  2025-05-17      3 2025-05-17


In [ ]:
from iglesia.utils import obtener_todos_los_textos

ModuleNotFoundError: No module named 'igles_ia'

In [ ]:
urls = {
    "Homilia": "https://www.vatican.va/content/leo-xiv/es/homilies/2025.html",
    "Angelus": "https://www.vatican.va/content/leo-xiv/es/angelus/2025.html",
    "Discurso": "https://www.vatican.va/content/leo-xiv/es/speeches/2025/may.index.html"
}


df = obtener_todos_los_textos(urls)
# si el título tiene menos de 10 caracteres, lo eliminamos
df = df[df["titulo"].str.len() > 10]
df = df.reset_index(drop=True)

# filename =f"{fecha_formateada}_{tipo_doc.replace(' ', '_').lower()}_{titulo_doc.replace(' ', '_').lower()}.json"

df["filename"] = df.apply(
    lambda x: f"{x['fecha']}_{x['tipo'].replace(' ', '_').lower()}_{x['titulo'].replace(' ', '_').lower()}",
    axis=1,
)
df

,tipo,fecha,titulo,url,texto,filename
0,Homilia,2025-05-18,V Domingo de Pascua – Celebración Eucarística ...,https://www.vatican.va/content/leo-xiv/es/homi...,CELEBRACIÓN EUCARÍSTICA CON MOTIVO DEL INICIO ...,2025-05-18_homilia_v_domingo_de_pascua_–_celeb...
1,Angelus,2025-05-18,"Regina Caeli, 18 de mayo de 2025",https://www.vatican.va/content/leo-xiv/es/ange...,PAPA LEÓN XIV\nSanta Misa de Inicio del Minist...,"2025-05-18_angelus_regina_caeli,_18_de_mayo_de..."
2,Discurso,2025-05-16,Al Cuerpo Diplomático (16 de mayo de 2025),https://www.vatican.va/content/leo-xiv/es/spee...,AUDIENCIA AL CUERPO DIPLOMÁTICO ACREDITADO ANT...,2025-05-16_discurso_al_cuerpo_diplomático_(16_...
3,Discurso,2025-05-15,A los Hermanos de las Escuelas Cristianas (15 ...,https://www.vatican.va/content/leo-xiv/es/spee...,DISCURSO DEL SANTO PADRE LEÓN XIV A LOS HERMAN...,2025-05-15_discurso_a_los_hermanos_de_las_escu...
4,Discurso,2025-05-14,A los participantes en el Jubileo de las Igles...,https://www.vatican.va/content/leo-xiv/es/spee...,DISCURSO DEL SANTO PADRE LEÓN XIV A LOS PARTIC...,2025-05-14_discurso_a_los_participantes_en_el_...
5,Discurso,2025-05-12,Audiencia del Santo Padre León XIV a los repre...,https://www.vatican.va/content/leo-xiv/es/spee...,DISCURSO DEL SANTO PADRE LEÓN XIV A LOS REPRES...,2025-05-12_discurso_audiencia_del_santo_padre_...
6,Homilia,2025-05-11,Homilía del Santo Padre León XIV en la cripta ...,https://www.vatican.va/content/leo-xiv/es/homi...,HOMILÍA DEL SANTO PADRE LEÓN XIV EN LA CRIPTA ...,2025-05-11_homilia_homilía_del_santo_padre_leó...
7,Angelus,2025-05-11,"Regina Caeli, 11 de mayo de 2025",https://www.vatican.va/content/leo-xiv/es/ange...,PAPA LEÓN XIV\nREGINA CAELI\nLogia Central de ...,"2025-05-11_angelus_regina_caeli,_11_de_mayo_de..."
8,Discurso,2025-05-10,Discurso del Santo Padre al Colegio Cardenalic...,https://www.vatican.va/content/leo-xiv/es/spee...,DISCURSO DEL SANTO PADRE LEÓN XIV AL COLEGIO C...,2025-05-10_discurso_discurso_del_santo_padre_a...
9,Homilia,2025-05-09,Santa Misa pro Ecclesia celebrada por el Roman...,https://www.vatican.va/content/leo-xiv/es/homi...,SANTA MISA PRO ECCLESIA CELEBRADA POR EL ROMAN...,2025-05-09_homilia_santa_misa_pro_ecclesia_cel...


In [3]:
from iglesia.agents import create_iglesia_content_crew

In [4]:

iglesia_content_crew = create_iglesia_content_crew(df[:2], LLM)

In [5]:
result = iglesia_content_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7bf45b52-54d2-4831-bcf8-ae50462afe97                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b2124eda-05b6-4650-892c-d38052fd2218
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: b2124eda-05b6-4650-892c-d38052fd2218
       Status: Executing Task...
    └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
            Status: In Progress

# Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
## Task: Analiza en profundidad el siguiente texto eclesiástico.
            Título: "V Domingo de Pascua – Celebración Eucarística por el inicio del ministerio petrino del obispo de Roma León XIV (18 de mayo de 2025)"
            Tipo de Documento: "Homilia"
            URL Original: "https://www.vatican.va/content/leo-xiv/es/homilies/2025/documents/20250518-inizio-pontificato.html"
            Texto: "CELEBRACIÓN EUCARÍSTICA CON MOTIVO DEL INICIO DEL MINISTERIO PETRINO DEL OBISPO DE ROMA LEÓN XIV
HOMILÍA DE SU SANTIDAD EL PAPA LEÓN XIV
Plaza de San Pedro Domingo 18 de mayo de 2025
[ Multimedia ]
___________________
Queridos hermanos cardenales, hermanos en el episcopado y en el sacerdocio, distinguidas autoridades y miembros del Cuerpo diplomático, ¡Saludos a los peregrinos que han venido al Jubileo de las Cofradías! hermanos y hermanas:
Los saludo a todos con el corazón lleno de gratitud,

🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
    Status: In Progress
└── 🧠 Thinking...

...

# Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
## Task: Resumen de los resumenes recibidos


# Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
## Final Answer: 
¡Bienvenidos al Resumen Semanal de Igles-IA! Esta semana, el Papa León XIV nos invita a reflexionar sobre la importancia del inicio del ministerio petrino, destacando que es una oportunidad para renovar nuestro compromiso de amor y servicio en la Iglesia, promoviendo la unidad, la paz y la fraternidad en el mundo. En su homilía, subraya que el amor de Dios, experimentado en lo personal, es la base para amar y servir a los demás, y que la misión de la Iglesia es ser fermento de reconciliación y respeto por la diversidad cultural.  

Por otro lado, en su Angelus, el Papa nos llama a orar por la paz en Gaza, Myanmar y Ucrania, resaltando la presencia espiritual del Papa Francisco y la importancia de María como mediadora de esperanza y pa

🚀 Crew: crew
├── 📋 Task: b2124eda-05b6-4650-892c-d38052fd2218
│      Assigned to: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│      Status: ✅ Completed
│   └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│           Status: ✅ Completed
├── 📋 Task: 00771583-504b-4321-97ab-2f2374cbe8f6
│      Assigned to: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│      Status: ✅ Completed
│   └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│           Status: ✅ Completed
└── 📋 Task: 9080e374-c583-4cc8-82a7-d3d99a39954e
       Status: Executing Task...
    └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: b2124eda-05b6-4650-892c-d38052fd2218
│      Assigned to: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│      Status: ✅ Completed
│   └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│           Status: ✅ Completed
├── 📋 Task: 00771583-504b-4321-97ab-2f2374cbe8f6
│      Assigned to: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│      Status: ✅ Completed
│   └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
│           Status: ✅ Completed
└── 📋 Task: 9080e374-c583-4cc8-82a7-d3d99a39954e
       Assigned to: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
       Status: ✅ Completed
    └── 🤖 Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9080e374-c583-4cc8-82a7-d3d99a39954e                                                                     │
│  Agent: Periodista católico, experto en analizar y estructurar textos del Papa y de la Iglesia                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7bf45b52-54d2-4831-bcf8-ae50462afe97                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [127]:
print(result)

{
  "fuente_documento": "A los Hermanos de las Escuelas Cristianas (15 de mayo de 2025)",
  "tipo_documento": "Discurso",
  "url_original": "https://www.vatican.va/content/leo-xiv/es/speeches/2025/may/documents/20250515-fratelli-scuole-cristiane.html",
  "resumen_general": "El Papa León XIV conmemora el tercer centenario de las Escuelas Cristianas, resaltando su historia de innovación educativa, su respuesta a los retos actuales de los jóvenes y la importancia del ministerio y misión laical en la enseñanza, promoviendo vocaciones y la santidad en la comunidad educativa.",
  "ideas_clave": [
    "Las Escuelas Cristianas nacieron como respuesta a las necesidades de los pobres, promoviendo una educación gratuita, inclusiva y adaptada a su tiempo.",
    "La innovación pedagógica de San Juan Bautista de La Salle, como el uso del idioma vernáculo y la participación familiar, sigue siendo relevante para afrontar los retos actuales.",
    "Los jóvenes de hoy enfrentan obstáculos como el indivi

In [115]:
crew_summary.kickoff()

╭─────────────────────────── Crew Execution Started ───────────────────────────╮
│                                                                              │
│  Crew Execution Started                                                      │
│  Name: crew                                                                  │
│  ID: c0be0261-cc23-4fd3-b889-d88162a9091c                                    │
│                                                                              │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 0587e000-d059-4ef0-babe-1e06002c0dd4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 0587e000-d059-4ef0-babe-1e06002c0dd4
       Status: Executing Task...
    └── 🤖 Agent: Text Summarizer
            Status: In Progress

# Agent: Text Summarizer
## Task: Create a summary of the following text: AUDIENCIA AL CUERPO 

CrewOutput(raw="In his recent addresses, Pope León XIV has articulated key themes reflecting the Church's mission and the role of education within society. On May 16, 2025, during his address to the diplomatic corps accredited to the Holy See, the Pope expressed deep gratitude to Ambassador George Poulides and acknowledged the support he received following his election. He placed emphasis on viewing the diplomatic community as a familial unit representing various peoples' diversity. He identified three foundational pillars of his ministry: peace as an active commitment, justice addressing global inequalities and upholding human dignity, and truth, which fosters genuine international relationships. The Pope urged collaboration among nations to tackle critical contemporary issues, including migration and environmental protection, calling for a collective effort to create a hopeful future, particularly for those in suffering (Source: Al Cuerpo Diplomático, 16 de mayo de 2025).\n\nIn a dif